# This notebook is used to verify the rotation correction in the latiss_cwfs_align script works as expected
## It first finds the pairs of CWFS images in the EFD
## It then fits the data a reports the zernikes

In [ ]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient, resample

%matplotlib inline

In [ ]:
def get_visitID_from_filename(filename):
    # Expects AT_O_20200218_000167.fits
    # parse out visitID from filename - this is highly annoying
    tmp=filename.split('_')
    prefix=tmp[2] # dayobs without the dashes
    suffix='{:05d}'.format(int(tmp[3].split('.')[0])) # SEQNUM, but need to trim extra 0 in obsid
    visitID = int((prefix+suffix))
    return visitID

In [ ]:
# # Temporarily needed to run at summit
# import os
# os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
# os.environ["OSPL_URI"] = "file:///home/patrickingraham/ospl.xml"

In [ ]:
efd_client = EfdClient('ncsa_efd') #summit_efd currently offline

Query for all the `endReadout` events on the timespan of the night.

In [ ]:
#2020031600092
t1 = Time("2020-03-17T00:59", format='isot', scale='tai')
t2 = Time("2020-03-17T01:28", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
end_readout = await efd_client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", 
                                           ["imageName", "exposureTime", "groupId", "imageType"], t1, t2)

Now match each entry. For each `i` item with `intra` in the name, there must be an `i+1` with `extra` otherwise it is not a pair. 

In [ ]:
intra_images = []
extra_images = []
intra_times = []
extra_times = []
intra_exptimes = []
extra_exptimes = []
nasmyth_pos = []
elevation_pos = []
hex_x = []
hex_y = []
#hex_z = []

i = 0
npairs = 0
nmiss = 0

while i < len(end_readout)-2:
    intra = end_readout['imageName'][i]
    extra = end_readout['imageName'][i+1]
    
    #skip known bad files
    if intra == 'AT_O_20200218_000179' and extra == 'AT_O_20200218_000180':
        i+=2
        continue
    
    # Check if consequetive pairs have the same groupID
    if (end_readout['groupId'][i] == end_readout['groupId'][i+1]) and (end_readout['groupId'][i+1] != end_readout['groupId'][i+2]) and (end_readout['imageType'][i] == 'ENGTEST'):
        print(f"Got a pair: {intra} x {extra}")
        intra_images.append(intra)
        extra_images.append(extra)
        intra_times.append(end_readout.index[i])
        extra_times.append(end_readout.index[i+1])
        intra_exptimes.append(end_readout['exposureTime'][i])
        extra_exptimes.append(end_readout['exposureTime'][i+1])

        npairs+=1
        # get required metadata that isn't in the header

        # include ability to correct for TAI if required, but set to zero for the moment
        offset37=0*TimeDelta(37.0, format='sec', scale='tai')
        t_start = Time(end_readout.index[i], scale='tai')-TimeDelta(end_readout.exposureTime[i], format='sec', scale='tai')+offset37
        t_end = Time(end_readout.index[i+1], scale='tai')+offset37

#         nasmyth_vals = await efd_client.select_packed_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", 
#                                              ["nasmyth2CalculatedAngle"], t_start , t_end)
        # Nasmyth rotator 2 values
        nasmyth_vals = await efd_client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", 
                                           ["nasmyth2CalculatedAngle1"], t_start , t_end) # , "private_rcvStamp"
        nasmyth_pos.append(nasmyth_vals.median()[0])
        
        # Elevation values
        elevation_vals = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                           ["elevationCalculatedAngle1"], t_start , t_end)
        elevation_pos.append(elevation_vals.median()[0])   
        
        # Hexapod values
        hexapod_vals = await efd_client.select_time_series("lsst.sal.ATHexapod.positionStatus", 
                                           ["reportedPosition0", "reportedPosition1", "reportedPosition2"], t_start , t_end)
        hex_x.append(hexapod_vals['reportedPosition0'].median())
        hex_y.append(hexapod_vals['reportedPosition1'].median())
        #hex_z.append(hexapod_vals['reportedPosition2'].median())
        
        i+=2
        
    else:
#         print(f"No Match: {intra} x {extra}")
        nmiss+=1
        i+=1

print(f"Got {npairs} pairs and {nmiss} misses.")

In [ ]:
#plt.hist(hexapod_vals['reportedPosition0'])
plt.plot(hexapod_vals['reportedPosition2'], '.')
#plt.plot(elevation_vals)

In [ ]:
#Time(Time(nasmyth_vals['private_rcvStamp'][0], format='unix', scale='tai')-Time(end_readout['groupId'][i], format='isot', scale='tai'), format='sec', scale='tai')

In [ ]:
# for p in range(len(intra_images)):
#     print((intra_images[p], extra_images[p], elevation_pos[p], nasmyth_pos[p]))

In [ ]:
# trim off files we don't want
# easiest to just get the times and modify search
#print(f'first time is {intra_times[0]}')
#print(f'last image time is {intra_times[8]}')

In [ ]:
import sys
#import asyncio
import logging
import time

In [ ]:
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign

In [ ]:
# Need to fake a DDS domain if at NCSA
import os
os.environ["LSST_DDS_DOMAIN"] = "asdfawefa"

In [ ]:
script = LatissCWFSAlign(index=1, remotes=False)
script.dataPath='/project/shared/auxTel/'

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
# script.filter='BG40'
# script.grating = 'empty_1'
# script.exposure_time = 30

In [ ]:
script.binning = 1
image_size= int(script.side * 2 / script.binning)

with_cross=True
if with_cross:
    print('Cross terms being used')
    script.sensitivity_matrix = [
        [1.0 / 161.0, 0.0, 0.0],
        [0.0, -1.0 / 161.0, (107.0/161.0)/4200],
        [0.0, 0.0, -1.0 / 4200.0]
]
#     script.sensitivity_matrix = [
#         [1.0 / 206.0, 0.0, 0.0],
#         [0.0, -1.0 / 206.0, (109.0/206.0)/4200],
#         [0.0, 0.0, -1.0 / 4200.0]
#         ]

else:
    print('No cross terms being used')
    script.sensitivity_matrix = [
        [1.0 / 161.0, 0.0, 0.0],
        [0.0, -1.0 / 161.0, 0.0],
        [0.0, 0.0, -1.0 / 4200.0],
    ]
    script.sensitivity_matrix = [
        [1.0 / 131.0, 0.0, 0.0],
        [0.0, -1.0 / 131.0, 0.0],
        [0.0, 0.0, -1.0 / 4200.0],
    ]
    

script.camera_rotation_angle=0

In [ ]:
# define structured array, but leave empty for the moment. Will get converted to pandas dataframe later
# intra, extra, elevation, N2_angle, hex_x, hex_y,meas_comaX, meas_comaY, meas_defocus, derot_comaX, derot_comaY, derot_defocus, hex_dx, hex_dy, hex_dz, dAz, dEl, drot?

In [ ]:
data = np.zeros(npairs, dtype=[('intraID', '<U35'), 
                                     ('extraID', '<U35'), 
                                     ('nas_pos', float), 
                                     ('el_pos', float), 
                                     ('cam_to_bore_ang', float),
                                     ('hex_x', float), 
                                     ('hex_y', float), 
                                     ('meas_comaX', float), 
                                     ('meas_comaY', float), 
                                     ('meas_defocus', float), 
                                     ('derot_comaX', float), 
                                     ('derot_comaY', float), 
                                     ('derot_defocus', float), 
                                     ('hex_dx', float),
                                     ('hex_dy', float),
                                     ('hex_dz', float),
                                     ('dAz', float),
                                     ('dEl', float),
                                     ('drot', float)
                              ])
images = np.zeros(npairs, dtype=[('intra_image', np.float64, (image_size, image_size)),
                                     ('extra_image', np.float64, (image_size, image_size)),
                                     ('image_mask', np.float64, (image_size, image_size)),
                                     ('zern_index', np.int16,(19)),
                                     ('measured_zerns', np.float64, (19))
                                    ] )

In [ ]:
#Need an enumeration for the data structure
start_time=time.time()
for n, (intra, extra, el_pos, nas_pos) in enumerate(zip(intra_images, extra_images, elevation_pos, nasmyth_pos)):
    print(intra,extra)
    script.intra_visit_id = get_visitID_from_filename(intra+'.fits')
    script.extra_visit_id = get_visitID_from_filename(extra+'.fits')
    script.angle = nas_pos - el_pos 


    results = await script.run_cwfs()
    
    # intra, extra, elevation, N2_angle, meas_comaX, meas_comaY, meas_defocus, derot_comaX, derot_comaY, derot_defocus, hex_dx, hex_dy, hex_dz, dAz, dEl, drot?
    data[n] = (script.intra_visit_id, 
               script.extra_visit_id, 
               nas_pos,
               el_pos,
               script.angle, # cam_to_bore_ang
               hex_x[n],
               hex_y[n],
               results['zerns'][0], # meas_comaX
               results['zerns'][1], # meas_comaY
               results['zerns'][2], # meas_defocus
               results['rot_zerns'][0], # derot_comaX
               results['rot_zerns'][1], # derot_comaY
               results['rot_zerns'][2], # derot_defocus
               results['hex_offset'][0], # hex_dx
               results['hex_offset'][1], # hex_dy
               results['hex_offset'][2], # hex_dz
               results['tel_offset'][0], # dAz
               results['tel_offset'][1], # dEl
               results['tel_offset'][2] # drot ?
              )
    images[n] = (script.I1[0].image0,
                 script.I2[0].image0,
                 script.algo.pMask,
                 np.arange(4,19+4),# zernike index
                 script.algo.zer4UpNm # zernike value
              )

end_time=time.time()
print('WFE fitting took {0:0.3f} seconds for {1} pairs'.format(end_time-start_time, n)) # 56.7s

In [ ]:
print('done')

In [ ]:
# plot the fits you want
n=2
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title("defocus 0.8 - intra")
ax11.imshow(images['intra_image'][n])
ax11.contour(images['image_mask'][n]) 
ax12 = fig1.add_subplot(122)
ax12.set_title("defocus 0.8 - extra")
ax12.imshow(images['extra_image'][n])
ax12.contour(images['image_mask'][n])

In [ ]:
# # print results
# script.show_results()
# # plot zernikes
# x = np.arange(9)+4
# plt.plot(x, script.algo.zer4UpNm[:9], 'o-', label=f'{script.dz}')
# xlim = plt.xlim()
# plt.plot(np.arange(15), np.zeros(15)+50, 'b--')
# plt.plot(np.arange(15), np.zeros(15)-50, 'b--')
# plt.xlim(xlim)
# plt.ylabel("Zernike coeff (nm)")
# plt.xlabel("Zernike index")
# plt.grid()
# plt.legend()

# print(script.algo.zer4UpNm[:9])

In [ ]:
# # plot image and mask
# fig1 = plt.figure(1, figsize=(12,8))
# ax11 = fig1.add_subplot(121)
# ax11.set_title("defocus 0.8 - intra")
# ax11.imshow(script.I1[0].image0)
# ax11.contour(script.algo.pMask) 
# ax12 = fig1.add_subplot(122)
# ax12.set_title("defocus 0.8 - extra")
# ax12.imshow(script.I2[0].image0)
# ax12.contour(script.algo.pMask) 

In [ ]:
# drop images from dataframe to make a pandas dataframe as it must be 1D
df = pd.DataFrame(data)

In [ ]:
filename="20200123_CWFS_verification_bin_"+str(script.binning)+'_sens_'+str(int(1/script.sensitivity_matrix[0][0]))+'_cross_terms_'+str(with_cross)+'_cam_rot_'+str(int(script.camera_rotation_angle))

In [ ]:
print(filename)

In [ ]:
#write to CSV file
#filename="20200123_CWFS_verification_nominal_settings_bin1_161_no_cross-terms-neg-angle"
#filename="20200123_CWFS_verification_nominal_settings_bin1_131_no_cross-terms-neg-angle"
#filename="20200123_CWFS_verification_nominal_settings_bin1_161_with_cross-terms-neg-angle"
#filename="20200123_CWFS_verification_nominal_settings_bin2_161_with_cross-terms-neg-angle"
#filename='test'
df.to_csv(filename+'.csv')
np.save(filename, images)

# Examine results of the rotation

In [ ]:
#df2 = pd.read_csv("20200123_CWFS_verification_angle_zero.csv")
#filename = "20200123_CWFS_verification_nominal_settings_bin1_161_with_cross-terms" # has bin=1, cross term, el-nas
#filename="20200123_CWFS_verification_nominal_settings_bin1_161_with_cross-terms-neg-angle"
#filename="20200123_CWFS_verification_nominal_settings_bin2_161_with_cross-terms-neg-angle"
#filename='test.csv'
df2 = pd.read_csv(filename+'.csv')
#df2['derot_defocus']=df2['derot_focus'] # needed because of error above in naming

In [ ]:
# images = np.load(filename+'.npy')
# # plot the fits you want
# n=5
# fig1 = plt.figure(1, figsize=(12,8))
# ax11 = fig1.add_subplot(121)
# ax11.set_title("defocus 0.8 - intra")
# ax11.imshow(images['intra_image'][n])
# ax11.contour(images['image_mask'][n]) 
# ax12 = fig1.add_subplot(122)
# ax12.set_title("defocus 0.8 - extra")
# ax12.imshow(images['extra_image'][n])
# ax12.contour(images['image_mask'][n])

In [ ]:
# Create plots wrt nasmyth position

# #dd_subplot(num_rows, num_cols, subplot_location
# mag=1.5
# ymin=-170 ; ymax=170
# xmin=-180 ; xmax=180

# fig = plt.figure(figsize=(6*mag,8*mag))
# ax = fig.add_subplot(2,2,1)
# ax.plot(df2['nas_pos'], df2['meas_comaX'], marker='x')
# ax.set_title('Measured')
# ax.set_xlabel('nasmyth_position [deg]')
# ax.set_ylabel('Coma-X [nm]')
# ax.set_ylim(ymin=ymin, ymax=ymax)
# ax.set_xlim(xmin=xmin, xmax=xmax)


# ax2 = fig.add_subplot(2,2,2)
# ax2.plot(df2['nas_pos'], df2['derot_comaX'], marker='x')
# ax2.set_title('Derotated')
# ax2.set_xlabel('nasmyth_position [deg]')
# #ax2.set_ylabel('derotated ComaX [nm]')
# ax2.set_ylim(ymin=ymin, ymax=ymax)
# ax2.set_xlim(xmin=xmin, xmax=xmax)

# ax3 = fig.add_subplot(2,2,3)
# ax3.plot(df2['nas_pos'], df2['meas_comaY'], marker='x')
# ax3.set_xlabel('nasmyth_position [deg]')
# ax3.set_ylabel('Coma-Y [nm]')
# ax3.set_ylim(ymin=ymin, ymax=ymax)
# ax3.set_xlim(xmin=xmin, xmax=xmax)


# ax4 = fig.add_subplot(2,2,4)
# ax4.plot(df2['nas_pos'], df2['derot_comaY'], marker='x')
# ax4.set_xlabel('nasmyth_position [deg]')
# ax4.set_ylim(ymin=ymin, ymax=ymax)
# ax4.set_xlim(xmin=xmin, xmax=xmax)

## Look at measured Zernikes as a function of instrument position
### Notably the difference in angle between the nasmyth and elevation

In [ ]:
#dd_subplot(num_rows, num_cols, subplot_location
mag=1.7
ymin=-170 ; ymax=170
xmin=-270 ; xmax=90

nrows=4 ; ncol=2
fig = plt.figure(figsize=(6*mag,8*mag))
ax = fig.add_subplot(nrows,2,1)
ax.plot(df2['cam_to_bore_ang'], df2['meas_comaX'], 'x')
ax.set_title('Measured')
#ax.set_xlabel('cam_to_bore_ang [deg]')
ax.set_ylabel('Coma-X [nm]')
ax.set_ylim(ymin=ymin, ymax=ymax)
ax.set_xlim(xmin=xmin, xmax=xmax)

ax2 = fig.add_subplot(nrows,2,2)
xvals=df2['cam_to_bore_ang']
yvals=df2['derot_comaX']
ax2.plot(xvals, yvals, 'x')
ax2.set_title('Derotated')
#ax2.set_xlabel('cam_to_bore_ang [deg]')
ax2.set_ylim(ymin=ymin, ymax=ymax)
ax2.set_xlim(xmin=xmin, xmax=xmax)
ax2.text(0.75,0.8,f"Mean {yvals.mean():.0f}", transform=ax2.transAxes)
ax2.text(0.75,0.9,f"Stdev {yvals.std():.0f}", transform=ax2.transAxes)

ax3 = fig.add_subplot(nrows,2,3)
ax3.plot(df2['cam_to_bore_ang'], df2['meas_comaY'], 'x')
#ax3.set_xlabel('cam_to_bore_ang [deg]')
ax3.set_ylabel('Coma-Y [nm]')
ax3.set_ylim(ymin=ymin, ymax=ymax)
ax3.set_xlim(xmin=xmin, xmax=xmax)

ax4 = fig.add_subplot(nrows,2,4)
xvals=df2['cam_to_bore_ang']
yvals=df2['derot_comaY']
ax4.plot(xvals, yvals, 'x')
#ax4.set_xlabel('cam_to_bore_ang [deg]')
ax4.set_ylim(ymin=ymin, ymax=ymax)
ax4.set_xlim(xmin=xmin, xmax=xmax)
ax4.text(0.75,0.8,f"Mean {yvals.mean():.0f}", transform=ax4.transAxes)
ax4.text(0.75,0.9,f"Stdev {yvals.std():.0f}", transform=ax4.transAxes)


ymin2=0 ; ymax2=250
xmin2=-270 ; xmax2=90

ax5 = fig.add_subplot(nrows,2,5)
ax5.plot(df2['cam_to_bore_ang'], np.sqrt(df2['meas_comaY']**2+df2['meas_comaX']**2), marker='x')
#ax5.set_xlabel('cam_to_bore_ang [deg]')
ax5.set_ylabel('RSS Coma X+Y [nm]')
ax5.set_ylim(ymin=ymin2, ymax=ymax2)
ax5.set_xlim(xmin=xmin2, xmax=xmax2)

ax6 = fig.add_subplot(nrows,2,6)
xvals=df2['cam_to_bore_ang']
yvals=np.sqrt(df2['derot_comaY']**2+df2['derot_comaX']**2)
ax6.plot(xvals, yvals, marker='x')
#ax6.set_xlabel('cam_to_bore_ang [deg]')
ax6.set_ylim(ymin=ymin2, ymax=ymax2)
ax6.set_xlim(xmin=xmin2, xmax=xmax2)
ax6.text(0.75,0.8,f"Mean {yvals.mean():.0f}", transform=ax6.transAxes)
ax6.text(0.75,0.9,f"Stdev {yvals.std():.0f}", transform=ax6.transAxes)

ax7 = fig.add_subplot(nrows,2,7)
ax7.plot(df2['cam_to_bore_ang'], df2['meas_defocus'], 'x')
ax7.set_xlabel('cam_to_bore_ang [deg]')
ax7.set_ylabel('Defocus [nm]')
ax7.set_ylim(ymin=ymin, ymax=ymax)
ax7.set_xlim(xmin=xmin, xmax=xmax)

ax8 = fig.add_subplot(nrows,2,8)
xvals=df2['cam_to_bore_ang']
yvals=df2['derot_defocus']
ax8.plot(xvals, yvals , 'x')
ax8.set_xlabel('cam_to_bore_ang [deg]')
ax8.set_ylim(ymin=ymin, ymax=ymax)
ax8.set_xlim(xmin=xmin, xmax=xmax)
ax8.text(0.75,0.8,f"Mean {yvals.mean():.0f}", transform=ax8.transAxes)
ax8.text(0.75,0.9,f"Stdev {yvals.std():.0f}", transform=ax8.transAxes)

## Look at suggested hexapod corrections as a function of instrument position
### Notably the difference in angle between the nasmyth and elevation

In [ ]:
# # for every 1mm of decenter, there is 107nm of WFE in defocus and 161 nm WFE in coma
# # or for every 1nm WFE in coma there is (107/161) nm of focus
# sensitivity_matrix = [
#         [1.0 / 161.0, 0.0, 0.0],
#         [0.0, -1.0 / 161.0, 0.0*(107.0/161.0)/4200],
#         [0.0, 0.0, -1.0 / 4200.0]
# ]

In [ ]:
#np.matmul(results['rot_zerns'], sensitivity_matrix)

In [ ]:
#dd_subplot(num_rows, num_cols, subplot_location
mag=1.3
ymin=-2 ; ymax=2  # um
xmin=-270 ; xmax=90

nrows=3 ; ncols=1
fig = plt.figure(figsize=(6*mag,8*mag))
ax = fig.add_subplot(nrows,ncols,1)
xvals=df2['cam_to_bore_ang']
yvals=df2['hex_dx']
ax.plot(xvals, yvals, marker='x')
ax.set_title('Suggested Hexapod Relative Offsets')
ax.set_ylabel('Hexapod dx [mm]')
ax.set_ylim(ymin=ymin, ymax=ymax)
ax.set_xlim(xmin=xmin, xmax=xmax)
ax.text(0.75,0.8,f"Mean {yvals.mean():.2f}", transform=ax.transAxes)
ax.text(0.75,0.9,f"Stdev {yvals.std():.2f}", transform=ax.transAxes)

ax2 = fig.add_subplot(nrows,ncols,2)
xvals=df2['cam_to_bore_ang']
yvals=df2['hex_dy']
ax2.plot(xvals, yvals, marker='x')
ax2.set_ylabel('Hexapod dy [mm]')
ax2.set_ylim(ymin=ymin, ymax=ymax)
ax2.set_xlim(xmin=xmin, xmax=xmax)
ax2.text(0.75,0.8,f"Mean {yvals.mean():.2f}", transform=ax2.transAxes)
ax2.text(0.75,0.9,f"Stdev {yvals.std():.2f}", transform=ax2.transAxes)

ymin2=-40 ; ymax2=40 #um

ax3 = fig.add_subplot(nrows,ncols,3)
xvals=df2['cam_to_bore_ang']
yvals=df2['hex_dz'] * 1000
ax3.plot(xvals, yvals, marker='x')
ax3.set_xlabel('cam_to_bore_ang [deg]')
ax3.set_ylabel('Hexapod dz [um]')
ax3.set_ylim(ymin=ymin2, ymax=ymax2)
ax3.set_xlim(xmin=xmin, xmax=xmax)
ax3.text(0.75,0.8,f"Mean {yvals.mean():.0f} [um]", transform=ax3.transAxes)
ax3.text(0.75,0.9,f"Stdev {yvals.std():.0f} [um]", transform=ax3.transAxes)

## Look at effect of binning data before fitting 

In [ ]:
filename1="20200123_CWFS_verification_nominal_settings_bin1_161_with_cross-terms-neg-angle"
filename2="20200123_CWFS_verification_nominal_settings_bin2_161_with_cross-terms-neg-angle"
#filename='test.csv'
df1 = pd.read_csv(filename1+'.csv')
df2 = pd.read_csv(filename2+'.csv')

#dd_subplot(num_rows, num_cols, subplot_location
ncols=2
nrows=4
mag=1.5
ymin=-10 ; ymax=10
xmin=-270 ; xmax=90

fig = plt.figure(figsize=(6*mag,8*mag))
ax = fig.add_subplot(nrows,ncols,1)
ax.plot(df2['cam_to_bore_ang'], df2['meas_comaX']-df1['meas_comaX'], marker='x')
ax.set_title('Measured')
#ax.set_xlabel('cam_to_bore_ang [deg]')
ax.set_ylabel('Coma-X [nm]')
ax.set_ylim(ymin=ymin, ymax=ymax)
ax.set_xlim(xmin=xmin, xmax=xmax)

ax2 = fig.add_subplot(nrows,ncols,2)
xvals=df2['cam_to_bore_ang']
yvals=df2['derot_comaX']-df1['derot_comaX']
ax2.plot(xvals, yvals, marker='x')
ax2.set_title('Derotated')
#ax2.set_xlabel('cam_to_bore_ang [deg]')
ax2.set_ylim(ymin=ymin, ymax=ymax)
ax2.set_xlim(xmin=xmin, xmax=xmax)
ax2.text(0.75,0.8,f"Mean {yvals.mean():.0f}", transform=ax2.transAxes)
ax2.text(0.75,0.9,f"Stdev {yvals.std():.0f}", transform=ax2.transAxes)


ax3 = fig.add_subplot(nrows,ncols,3)
ax3.plot(df2['cam_to_bore_ang'], df2['meas_comaY']-df1['meas_comaY'], marker='x')
#ax3.set_xlabel('cam_to_bore_ang [deg]')
ax3.set_ylabel('Coma-Y [nm]')
ax3.set_ylim(ymin=ymin, ymax=ymax)
ax3.set_xlim(xmin=xmin, xmax=xmax)

ax4 = fig.add_subplot(nrows,ncols,4)
xvals = df2['cam_to_bore_ang']
yvals = df2['derot_comaY']-df1['derot_comaY']
ax4.plot(xvals, yvals, marker='x')
#ax4.set_xlabel('cam_to_bore_ang [deg]')
ax4.set_ylim(ymin=ymin, ymax=ymax)
ax4.set_xlim(xmin=xmin, xmax=xmax)
ax4.text(0.75,0.8,f"Mean {yvals.mean():.0f}", transform=ax4.transAxes)
ax4.text(0.75,0.9,f"Stdev {yvals.std():.0f}", transform=ax4.transAxes)

ymin2=-10 ; ymax2=10
xmin2=-270 ; xmax2=90

ax5 = fig.add_subplot(nrows,ncols,5)
ax5.plot(df2['cam_to_bore_ang'], np.sqrt(df2['meas_comaY']**2+df2['meas_comaX']**2)-np.sqrt(df1['meas_comaY']**2+df1['meas_comaX']**2), marker='x')
#ax5.set_xlabel('cam_to_bore_ang [deg]')
ax5.set_ylabel('RSS Coma X+Y [nm]')
ax5.set_ylim(ymin=ymin2, ymax=ymax2)
ax5.set_xlim(xmin=xmin2, xmax=xmax2)

ax6 = fig.add_subplot(nrows,ncols,6)
xvals=df2['cam_to_bore_ang']
yvals=np.sqrt(df2['derot_comaY']**2+df2['derot_comaX']**2)-np.sqrt(df1['derot_comaY']**2+df1['derot_comaX']**2)
ax6.plot(xvals, yvals, marker='x')
#ax6.set_xlabel('cam_to_bore_ang [deg]')
ax6.set_ylim(ymin=ymin2, ymax=ymax2)
ax6.set_xlim(xmin=xmin2, xmax=xmax2)
ax6.text(0.75,0.8,f"Mean {yvals.mean():.0f}", transform=ax6.transAxes)
ax6.text(0.75,0.9,f"Stdev {yvals.std():.0f}", transform=ax6.transAxes)

ax7 = fig.add_subplot(nrows,ncols,7)
ax7.plot(df2['cam_to_bore_ang'], df2['meas_defocus']-df1['meas_defocus'], marker='x')
ax7.set_xlabel('cam_to_bore_ang [deg]')
ax7.set_ylabel('Defocus [nm]')
ax7.set_ylim(ymin=ymin, ymax=ymax)
ax7.set_xlim(xmin=xmin, xmax=xmax)

ax8 = fig.add_subplot(nrows,ncols,8)
xvals = df2['cam_to_bore_ang']
yvals = df2['derot_defocus']-df1['derot_defocus']
ax8.plot(xvals, yvals, marker='x')
ax8.set_xlabel('cam_to_bore_ang [deg]')
ax8.set_ylim(ymin=ymin, ymax=ymax)
ax8.set_xlim(xmin=xmin, xmax=xmax)
ax8.text(0.75,0.8,f"Mean {yvals.mean():.0f}", transform=ax8.transAxes)
ax8.text(0.75,0.9,f"Stdev {yvals.std():.0f}", transform=ax8.transAxes)

In [ ]:
#write to CSV file
df2.to_csv("20200123_match_zerns.csv")

# Fit the data

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def parabola(x,b, x0, a):
    return b + a*(x-x0)**2 
def line(x,b, m):
    return b + m*x 
def invparabola(y,b,x0,a):
    return x0+np.sqrt((y-b)/a)
def invline(y,b,m):
    return (y-b)/m

In [ ]:
#df3 = pd.DataFrame.from_csv("20200123_match_zerns.csv")
# use below for pandas 1.0+
df3 = pd.read_csv("20200123_match_zerns.csv")

In [ ]:
df3

### Plot Y-Coma as a function of Y-hexapod decentering

In [ ]:
inds = np.arange(0,10) # drop the last value
inds = [0,1,2,3,5,6,7,8]

xdata=df3['y'][inds]
ydata=df3['zern_ycoma_nm'][inds]

plt.plot(xdata,ydata,'o')
x=np.arange(np.min(xdata), np.max(xdata), np.abs(np.max(xdata) - np.min(xdata))/100 )
popt,pcov = curve_fit(line, xdata, ydata)
print('Wavefront X-Coma as a function of Hexapod displacement',popt)

plt.plot(x,line(x, *popt))
# plt.xlabel('Hexapod displacement in the plane')
plt.ylabel('Zernike Coefficient in nm')
plt.title('Wavefront Y-Coma as a function of hexapod Y-displacement')
plt.show()

print(popt)

In [ ]:
(240+200)/ (-6 - -3)

### Plot Defocus as a function of Y-hexapod decentering

In [ ]:
#Coma as a function of x,y offset (8-23, 31-35)
inds = np.arange(0,11)
inds = [0,1,2,3,5,6,7,8]

xdata=df3['y'][inds]
ydata=df3['zern_defocus_nm'][inds]

plt.plot(xdata,ydata,'o')
x=np.arange(np.min(xdata), np.max(xdata), np.abs(np.max(xdata) - np.min(xdata))/100 )
popt,pcov = curve_fit(line, xdata, ydata)
print('Wavefront X-Coma as a function of Hexapod displacement',popt)

plt.plot(x,line(x, *popt))
# plt.xlabel('Hexapod displacement in the plane')
plt.ylabel('Zernike Coefficient in nm')
plt.title('Wavefront Defocus as a function of y-hexapod displacement')
plt.show()

print(popt)